# Nueral Network From scratch

---

## Deep Neural Network

In [222]:
class Layer:
    """
    Layer in Network
    
    Parameters
    units: number of neurons in a layer
    activation: activation function for the layer
    """
    def __init__(self,units,activation):
        """constructor for Layer class"""
        self.units=units
        self.activation=activation
        self.weights=None
        self.bias=None
        self.Z=None
        self.dZ=None
        self.Input=None
        self.weights_temp=None
        self.bias_temp=None
        import numpy as np #import numpy package
    
    def initParams(self,input_shape):
        """initialize weights and bias"""
        self.weights=(np.random.randn(self.units,input_shape[0]).round(2))*0.01
        self.bias=np.zeros(self.units).reshape(-1,1)
    
    def getActivation(self,input_matrix):
        """Compute activation for a layer"""
        Z=(np.matmul(self.weights,input_matrix))+self.bias 
        self.Z=Z #cache Z
        self.Input=input_matrix #record input to layer
        if (self.activation=='relu'):
            A=self.ReLU(Z)
        elif (self.activation=='sigmoid'):
            A=self.Sigmoid(Z)
        elif (self.activation=='linear'):
            A=self.Linear(Z)
        return A
    
    #Activation functions
    def ReLU(self,Z):
        """Rectified Linear activation function"""
        temp=Z.copy()
        temp[temp<=0]=0
        return temp
    def Sigmoid(self,Z):
        """Sigmoid activation function"""
        return 1/(1+np.exp(-Z))
    def Linear(self,Z):
        """Linear activation function"""
        return Z

In [337]:
class Network:
    run=0 #gradient descent not yet run
    """
    Network of Layers
    
    Parameters
    layers: List of tuples. Each tuple represents a layer.\
    Each tuple contains number of units and activation for the layer.
    
    """
    def __init__(self,layers):
        """Constructor for Network class"""
        self.layers=layers
        self.Layer_objs=[]
        

    def forwardProp(self,A):
        """Implementation of forward propagation"""
        if (Network.run==0): #first run of gradient descent
            for layer in self.layers:
                Layer_obj=Layer(layer[0],layer[1])
                Layer_obj.initParams(A.shape)
                A=Layer_obj.getActivation(A)
                self.Layer_objs.append(Layer_obj)
            Network.run=1 #gradient descent already run
            return A
        else: #other runs of gradient descent
            for obj in self.Layer_objs:
                obj.Input=A #assign value of Input variable
                A=obj.getActivation(A)
            return A
                           
    
    def backProp(self, A, Y, alpha=0.1):
        """Implementation of back propagation"""
        n=len(self.Layer_objs)
        m=A.shape[1]
        for i in range(-1,-(n+1),-1):
            current_layer=self.Layer_objs[i] #layer being indexed
            if (i==-1):
                dZ_current_layer=A-Y
                current_layer.dZ=dZ_current_layer
            else: 
                layer_above=self.Layer_objs[i+1]
                W_layer_above=layer_above.weights #obtain weight matrix of layer above
                dZ_layer_above=layer_above.dZ #obtain dz for layer above
                
                #fetch differential of activation
                if (current_layer.activation=='relu'):
                    diff_activation=self.diffReLU(current_layer.Z)
                elif (current_layer.activation=='sigmoid'):
                    diff_activation=self.diffSigmoid(current_layer.Z)
                elif (current_layer.activation=='linear'):
                    diff_activation=self.diffLinear(current_layer.Z)
                    
                dZ_current_layer=(np.matmul(W_layer_above.T,dZ_layer_above)*diff_activation)
                current_layer.dZ=dZ_current_layer
            dW=np.matmul(dZ_current_layer,current_layer.Input.T)/m
            db=np.sum(dZ_current_layer,axis=1,keepdims=True)/m
            
            current_layer.weights_temp=current_layer.weights-(alpha*dW)
            current_layer.bias_temp=current_layer.bias-(alpha*db)
            
        #update weights and bias vectors
        for obj in self.Layer_objs:
            obj.weights=obj.weights_temp
            obj.bias=obj.bias_temp
            
    #differentiate activation functions
    def diffReLU(self,z):
        """differentiate relu activation function"""
        temp=z.copy()
        temp[temp<=0]=0
        temp[temp>0]=1
        return temp
    def diffSigmoid(self,z):
        """differentiate sigmoid activation function"""
        a=1/(1+np.exp(-z))
        return a*(1-a)
    def diffLinear(self,z):
        """differentiate linear activation function"""
        m,n=z.shape
        return np.ones((m,n))
    
    
    def gradientDescent(self, n_iter, X, Y, alpha):
        """Run gradient descent algorithm"""
        for i in range(n_iter):
            A=self.forwardProp(X)
            print(f'Iteration: {i+1}\tCost: {self.logLoss(A,Y)}')
            self.backProp(A,Y,alpha)
        Network.run=0
        return
            
            
    def logLoss(self,A,Y):
        """Calculate cost"""
        m=A.shape[1]
        loss=-((Y*np.log(A))+((1-Y)*np.log(1-A)))
        cost=(np.sum(loss))/m
        return cost
            
        